In [15]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib.ticker import AutoMinorLocator
# from scipy.optimize import curve_fit
# import os
import subprocess



In [16]:
# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
mass_nucleon = 0.938273
mass_C12= 39.96259098
alpha_fine = 1/137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

# map from data set index to data set name (author, year, etc.)
# dataSet_to_name = {1:"Barreau:1983ht", 2:"O'Connell:198", 3:"Sealock:1989nx", 4:"Baran:1988tw", 5:"Bagdasaryan:1988hp", 6:"Dai - HallA:2019da", 
#     7:"Arrington:1995hs", 8:"Day:1993md", 9:"Arrington:1998psnoCC", 10:"Gaskell:2008", 11:"Whitney:1974hr", 12:"AlsamiJan05", 13:"VaheJun07", 
#     14:"Gomez74", 15:"Fomin", 16:"Yamaguchi73", 17:"Ryan84", 18:"Cyzyk:1963zz", 19:"Bounin63", 20:"Photo-Daphne", 21:"Spamer70", 22:"Goldemberg64", 
#     23:"DeForrest65", 24:"Donnelly68", 25:"Barreau:French",26:"Julia:Genie",27:"Antony-Spies:1970jjs"}


#dataSet_to_name = {1:"Meziani:1984is_90deg", 2:"Meziani:1984is_140deg", 3:"Williamson:1997_140deg", 4:"Williamson:1997_90deg", 5:"Meziani:1984is_60deg", 6:"Williamson_45.5deg", 7:"Whitney:1974hr_60deg", 9:"Torizuka1975"}
#dataSet_to_name = {3:"Williamson:1997_140deg", 4:"Williamson:1997_90deg", 6:"Williamson_45.5deg", 10:"Meziani_0.120_140", 11:"Meziani_0.160_140", 12:"Meziani_0.200_140", 13:"Meziani_0.240_140", 14:"Meziani_0.280_140", 
#                   15:"Meziani_0.320_140", 16:"Meziani_0.360_140", 17:"Meziani_0.400_140", 18:"Meziani_0.440_140", 19:"Meziani_0.480_140",
#                   20:"Meziani_0.120_90", 21:"Meziani_0.160_90", 22:"Meziani_0.200_90", 23:"Meziani_0.240_90", 24:"Meziani_0.280_90",
#                  25:"Meziani_0.320_90", 26:"Meziani_0.360_90", 27:"Meziani_0.400_90", 30:"Torizuka:1975_Meziani_combined", 31:"Whitney:1974hr_60deg"}
#updated as of 04/02
dataSet_to_name = {1:"Meziani0.120_90", 2:"Meziani0.120_140", 3:"Williamson0.130_140", 4:"Torizuka0.15_35", 5:"Williamson0.150_90", 6:"Meziani0.160_60", 7:"Meziani0.160_90", 8:"Meziani0.160_140", 9:"Williamson0.160_140 ", 10:"Torizuka0.183_35", 
                   11:"Torizuka0.183_45", 12:"Torizuka0.183_55", 13:"Williamson0.189_140", 14:"Mezian0.2_60", 15:"Mezini0.2_90", 16:"Williamson0.2_90", 17:"Meziani0.2_140", 18:"Wlliamson_0.219_140", 19:" Meziani0.24_60", 20:"Meziani0.24_90",
                   21:"Meziani0.24_140", 22:"Williamson0.248_90", 23:"Williamson0.249_140", 24:"Torizuka0.25_50", 25:"Torizuka0.25_60", 26:"Torizuka0.25_70", 27:"Meziani0.28_60", 28:"Willaimson0.28_90", 29:"Meziani0.28_140", 30:"Williamson0.287_140",
                   31:"Williamson0.298_90", 32:"Meziani0.32_90", 33:"Meziani0.32_140", 34:"Williamson0.327_140", 35:" Meziani0.34_60", 36:"Williamson0.347_90", 37:" Williamson0.348_45.5", 38:" Meziani0.36_90", 39:"Meziani0.36_140", 40:"Williamson0.367_140",
                   41:"Williamson0.372_90", 42:"Meziani0.4_60", 43:"Meziani0.4_90", 44:"Meziani0.4_140", 45:"Williamson0.408_45.5", 46:"Meziani0.44_140", 47:" Williamson0.471_45.5", 48:"Meziani0.48_60", 49:"Meziani0.48_140", 50:"Whitney0.5_60",
                   51:"Williamson0.545_45.5", 52:"Meziani0.56_60", 53:"Williamson0.628_45.4", 54:"Williamson0.681_45.5", 55:"Williamson0.739_45.5", 56:"Williamson0.782_45.5", 57:"Williamson0.841_45.5"}


#updated normalization values for 40Ca
#reference values for normlization
#dataSet_to_normalization = {1:0.97, 2:0.93, 3:1.0, 4:1.00 , 5:1.1 , 6:1.0 , 7:1.059}

#updated normalization values as of 02/26/2025
#dataSet_to_normalization = {1:0.967103118, 2:0.965862545, 3:1.0, 4:1.0 , 5:0.9, 6:1.0 , 7:1.4, 9:0.9}

#updated normalization values as of 03/10/2025
#dataSet_to_normalization = {3:1.0, 4:1.0, 6:1.0, 10:1.080, 11:0.989, 12:0.937, 13:0.899, 14:0.913, 
#                   15:0.939, 16:0.976, 17:0.983, 18:0.970, 19:1.001,
#                   20:1.094, 21:0.934, 22:0.970, 23:0.870, 24:0.903
#                   25:1.036, 26:0.959, 27:0.927, 30:1.00, 31:0.99}

#04/03/2025
dataSet_to_normalization ={1:1.094, 2:1.080, 3:1.0, 4:1.051 , 5:1.0 , 6:0.964 , 7:0.934 ,8:0.989 , 9:1.0, 10:1.051, 11:1.051 ,
                           12:1.051 , 13:1.0, 14:0.964 , 15:0.970 , 16:1.0, 17:0.937 , 18:1.0, 19:0.964 , 20:0.870 , 21:0.899 ,
                           22:1.0 , 23:1.0 , 24:1.051 , 25:1.051 , 26:1.051 , 27:0.964 , 28:1.0 , 29:0.913 , 30:1.0 , 31:1.0 ,
                           32:1.036 , 33:0.939 , 34:1.0 , 35:0.964 , 36:1.0 , 37:1.0 , 38:0.959 , 39:0.976 , 40:0.976 , 41:1.0 ,
                           42:0.964 , 43:0.927 , 44:0.983 , 45:1.0 , 46:0.970 , 47:1.0 , 48:0.964 , 49:1.001 , 50:0.954 , 51:1.0 ,
                           52:1.0 , 53:1.0 , 54:1.0 , 55:1.0 , 56:1.0 , 57:1.0 }


#leave 1,5 at the end for fixing the  a

# map from data set index to normalization error
# dataSet_to_normError = {1:0.24320E-02, 2:0.86008E-02, 
#                         # 3:0.48305E-02,
#                          3:0.1,
#                           4:0.45501E-02, 5:0.82797E-02, 6:0.52969E-02, 7:0.13229E-01, 8:0.33307E-02, 
#         9:0.34432E-02, 10:0.50742E-02, 11:0.15258E-01, 12:0.67079E-03, 13:0.69905E-03, 14:0.14875E-01, 15:0.30856E-02, 16:0.29024E-02, 
#         17:0.13049E-01, 
#         18:0.2, 
#         19:0.23, 20:0.10513, 21:0.4, 22:0.1, 23:0.1, 24:0.1, 25:0.002432, 26:0.0, 27:0.0}

#updated normalization error values for 40Ca
dataSet_to_normError = {1:0.037, 2:0.037, 3:0.037, 4:0.037 , 5:0.037 , 6:0.037 , 7:0.037 ,8:0.037 , 9:0.037, 10:0.037, 11:0.037 ,
                        12:0.037 , 13:0.037, 14:0.037 , 15:0.037 , 16:0.037, 17:0.037 , 18:0.037, 19:0.037 , 20:0.037 , 21:0.037 ,
                        22:0.037*2 , 23:0.037 , 24:0.037 , 25:0.037 , 26:0.037 , 27:0.037 , 28:0.037 , 29:0.037 , 30:0.037 , 31:0.037 ,
                        32:0.037 , 33:0.037 , 34:0.037 , 35:0.037 , 36:0.037 , 37:0.037 , 38:0.037 , 39:0.037 , 40:0.037 , 41:0.037 ,
                        42:0.037 , 43:0.037 , 44:0.037 , 45:0.037 , 46:0.037 , 47:0.037 , 48:0.037 , 49:0.037 , 50:0.037 , 51:0.037 ,
                        52:0.037 , 53:0.037 , 54:0.037 , 55:0.037 , 56:0.037 , 57:0.037}





In [17]:
# utility functions:

# round to only 3 significant figures (for printing)
def round_sig_3(x):
    return '%s' % float('%.3g' % x)

# append a row to a dataframe
def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

# linear model for Rosenbluth fitting
def linear_model(x, a, b):
    return a * x + b

# prepare the dataframe
# "filepath" should be the path to the csv file that contains columns {'A', 'Z', 'nu', 'E0', 'ThetaDeg', 'cross', 'error', 'dataSet'}
def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    # calculate normalized cross section:
    df["cross"] = df["cross"]
    df["normalization"]=df["dataSet"].map(dataSet_to_normalization)
    df["normError"]=df["dataSet"].map(dataSet_to_normError)
    df['normCross'] = df['cross'] * df['normalization']
    system_err = 0.00
    df['error'] = np.sqrt(df['error']**2 + ((system_err*df['cross'])**2))
    df['normCrossError']=df['normCross']*np.sqrt((df['error']/df['cross'])**2+(df['normError']/df['normalization'])**2)
    
    # calculate the kinematic variables:
    df["ThetaRad"]=df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"]=(np.sin(df["ThetaRad"]/2))**2
    df["cos2(T/2)"]=(np.cos(df["ThetaRad"]/2))**2
    df["tan2(T/2)"]=(np.tan(df["ThetaRad"]/2))**2

    df["nu_elastic"]=df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12)
    df["Ex"]=df["nu"]-df["nu_elastic"]
    df["Ex"] = df["nu"] - (df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12))

    # R = 2.894, Z = 20, A = 40
    df["R"]=1.1*(df["A"])**(1/3)+0.86/((df["A"])**(1/3))
    df["Veff"]=0.0074
    df["Eeff"]=df["E0"]+df["Veff"]
    df["Eprime"]=df["E0"]-df["nu"]
    df["Eprime_eff"]=df["Eprime"]+df["Veff"]

    df["F2foc"]=(df["Eeff"]/df["E0"])**2
    df["Q2"]=4*df["E0"]*(df["Eprime"])*df["sin2(T/2)"]
    df["Q2eff"]=4*df["Eeff"]*df["Eprime_eff"]*df["sin2(T/2)"]
    df["q3momt_squared"]=df["nu"]**2+df["Q2eff"]
    df["q3momt"]=np.sqrt(df["q3momt_squared"])

    df["W2"]=mass_nucleon**2+2*mass_nucleon*df["nu"]-df["Q2eff"]
    df["epsilon"]=1/(1+2*(1+(df["nu"]**2)/df["Q2eff"])*df["tan2(T/2)"]) 

    df["gamma"]=alpha_fine*df["Eprime_eff"]*(df["W2"]-mass_nucleon**2)/((4*((np.pi)**2)*df["Q2eff"]*mass_nucleon*df["E0"])*(1-df["epsilon"]))
    df["Sig_R"]=df["normCross"]/df["gamma"]
    df["D_sig_R"]=df["error"]/df["gamma"]
    df["Sig_mott"]=4*(alpha_fine**2)*(df["Eprime"]**2)*df["cos2(T/2)"]/(df["Q2"]**2)
    df["Sig_mott_eff"]=df["Sig_mott"]*df["E0"]/df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"]=(df["q3momt_squared"]**2)/(4*(alpha_fine**2)*(df["Eprime_eff"]**2)*(df["cos2(T/2)"]+2*(df["q3momt_squared"]/df["Q2eff"])*df["sin2(T/2)"]))
    df["Hcc"]=df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"]=df["H"]*df["normCross"]
    df["Hstar_error(nb)"]=df["H"]*df["normCrossError"]
    df["Hstar_Sig(GeV)"]=df["Hstar_Sig(nb)"]/((0.1973269**2)*10000000)
    df["Hstar_error(GeV)"]=df["Hstar_error(nb)"]/((0.1973269**2)*10000000)
    df["Hcc_Sig(nb)"]=df["Hcc"]*df["normCross"]
    df["Hcc_error(nb)"]=df["Hcc"]*df["normCrossError"]
    df["Hcc_Sig(GeV)"]=df["Hcc_Sig(nb)"]/((0.1973269**2)*10000000)
    df["Hcc_error(GeV)"]=df["Hcc_error(nb)"]/((0.1973269**2)*10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # apply the new condition for datasets 1, 2, and 5
    mask = df['dataSet'].isin([1, 2, 5])
    df.loc[mask & (df['Eprime'] < 0.2), 'normError'] = 0.037 + (0.2 - df['Eprime']) * 17
    df.loc[mask & (df['Eprime'] > 0.2), 'normError'] = 0.037
    
    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"]=pd.cut(x=df["q3momt"],bins=qvbins,labels=qvbin_names,right=True)
    df["qvcenter"]=pd.cut(x=df["q3momt"],bins=qvbins,labels=qvcenters,right=True)
    df['qvcenter']=pd.to_numeric(df['qvcenter'])
    df["Q2bin"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2bin_names,right=True)
    df["Q2center"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2centers,right=True)
    df['Q2center']=pd.to_numeric(df['Q2center'])
    # now every row of data has a bin (Q2/qv) label

    return df

# the entire dataset is firstly subdivided into bins of qv, then further divided into sub-bins of nu.
# "nu_grid" is essentially the edges of nu sub-bins
def customized_nu_grid(bin_index):
    step_settings = np.array([
        [0.0007,  0.001,  0.002, 0.05],
        [0.0002,  0.001,  0.002, 0.05],
        [0.0002,  0.001,  0.002, 0.05],
        [0.00025, 0.0045, 0.02,  0.05],
        [0.00025, 0.004,  0.02,  0.05],
        [0.0004,  0.0055, 0.008, 0.05],
        [0.006,   0.006,  0.01,  0.01],
        [0.01,    0.01,   0.03,  0.05],
        [0.01,  0.01,  0.05,  0.05],# bad: 8: 0.57
        [0.01,  0.01,  0.01,  0.013],
        [0.01,  0.01,  0.05,  0.05], # bad: 10: 0.756
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01],
        [0.01,  0.01,  0.01,  0.01]
    ])

    
    qvcenter = qvcenters[bin_index]
    Ex_nu50 = 0.05-(qvcenter**2-0.05**2)/(2*mass_C12)
    W2_nu50 = mass_nucleon**2+2*mass_nucleon*0.05-(qvcenter**2-0.05**2)
    W2_nuQvcenter = mass_nucleon**2+2*mass_nucleon*qvcenter
    W2_nu0 = mass_nucleon**2-(qvcenter**2)
    if qvcenter <= 0.991:
        Ex_grid = np.arange(0.0,Ex_nu50, Ex_nu50/60)
        W2_grid = np.arange(W2_nu50,W2_nuQvcenter,(W2_nuQvcenter-W2_nu50)/60)
        nu_grid = np.concatenate([np.sqrt(mass_C12**2+qvcenter**2+2*mass_C12*Ex_grid)-mass_C12,
                                  np.sqrt(qvcenter**2+W2_grid)-mass_nucleon])
    else:
        W2_grid = np.arange(W2_nu0,W2_nuQvcenter,(W2_nuQvcenter-W2_nu0)/100)
        nu_grid = np.sqrt(qvcenter**2+W2_grid)-mass_nucleon
    # step_setting = step_settings[bin_index]
    # W_peaks = np.array([0.93,1.07,1.23]) 
    # W_locations = 0.025-mass_nucleon+np.sqrt(qvcenter**2+W_peaks**2)
    # EX_grid = np.arange(0.0,W_locations[0], step_setting[0])
    # QE_grid = np.arange(W_locations[0],W_locations[1],step_setting[1])
    # DL_grid = np.arange(W_locations[1],W_locations[2],step_setting[2])
    # after_grid = np.arange(W_locations[2],qvcenter,step_setting[3])
    # if qvcenter == 0.475:
    #     EX_grid = np.arange(0.0,W_locations[0], step_setting[0])
    #     QE_grid = np.arange(W_locations[0],W_locations[1],step_setting[1])
    #     DL_grid = np.arange(W_locations[1],0.35,step_setting[2])
    #     after_grid = np.arange(0.35,qvcenter,step_setting[3])
    # nu_grid = np.concatenate([EX_grid,QE_grid,DL_grid,after_grid])

    return nu_grid

# RLRT plot qvbins, investigate individual bin, combining different step sizes
def RLRT_extract_and_plot(df=None,bin_index=0,Rosenbluth_plot=False):

    # plot height settings
    RL_plot_heights = [0.05,0.07,0.08,0.10,0.11,0.05,0.0440, 0.0175, 0.0150,0.01, 0.01, 0.015,0.1,0.12,0.08,0.05,0.04,0.0125]
    RT_plot_heights = [0.013,0.027,0.03,0.035,0.041,0.044,0.045,0.045,0.049,0.04,0.042,0.045,0.045,0.035,0.035,0.026,0.015,0.012]

    
    fig, axs = plt.subplots(1, 2, figsize=(10, 4),dpi=300)
    qvcenter = qvcenters[bin_index]


    
    qv2center = qvcenter**2
    bin_data = df.loc[df['qvcenter']==qvcenter].copy()
    fit = pd.DataFrame()

    nu_grid = customized_nu_grid(bin_index)
    Q2_grid = qvcenter**2-nu_grid**2
    Ex_grid = nu_grid-Q2_grid/(2*mass_C12)
    W2_grid = mass_nucleon**2+2*mass_nucleon*nu_grid-Q2_grid
   
    
    # loop through nu_grid
    for j in range(len(nu_grid)-1):
        nu = nu_grid[j]
        Q2 = Q2_grid[j]
        Ex = Ex_grid[j]
        W2 = W2_grid[j]

        if qvcenter <= 0.991:
            if nu+(nu_grid[j+1]-nu)/2 <= 0.05:
                picked = bin_data.loc[(bin_data['Ex']>=Ex_grid[j]) & (bin_data['Ex']<Ex_grid[j+1])].copy()
                picked['Hbc_Sig(GeV)'] = picked['bc_qv_ex']*picked['Hcc_Sig(GeV)']
                picked['Hbc_error(GeV)'] = picked['bc_qv_ex']*picked['Hcc_error(GeV)']
            else:
                picked = bin_data.loc[(bin_data['W2']>=W2_grid[j]) & (bin_data['W2']<W2_grid[j+1])].copy()
                picked['Hbc_Sig(GeV)'] = picked['bc_qv_w2']*picked['Hcc_Sig(GeV)']
                picked['Hbc_error(GeV)'] = picked['bc_qv_w2']*picked['Hcc_error(GeV)']      
        else:
            picked = bin_data.loc[(bin_data['W2']>=W2_grid[j]) & (bin_data['W2']<W2_grid[j+1])].copy()
            picked['Hbc_Sig(GeV)'] = picked['bc_qv_w2']*picked['Hcc_Sig(GeV)']
            picked['Hbc_error(GeV)'] = picked['bc_qv_w2']*picked['Hcc_error(GeV)']      

        x = np.array(picked["epsilon"].values)
        y = np.array(picked["Hbc_Sig(GeV)"].values)
        y_err = np.array(picked["Hbc_error(GeV)"].values)
        
        if len(y)>2 and (np.max(x)-np.min(x))>=0.25:
            #____________________________________ start Rosenbluth fit _______________________________________________________
            
            if len(y)==2:
                x = np.concatenate([x, x])
                y = np.concatenate([y+0.1*y_err, y-0.1*y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])

            params, covariance = curve_fit(linear_model, x, y, sigma=y_err, absolute_sigma=True)

            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            Chi2 = np.sum(np.square((y-linear_model(x,a_opt,b_opt))/y_err))
            RL = a_opt/1000
            RLerr = a_err/1000 
            RT = (2*b_opt*q2/(qvcenter**2))/1000
            RTerr = (2*b_err*q2/(qvcenter**2))/1000
            new_row = pd.Series({'qvcenter':qvcenter,'nu':nu_mid,'RL':RL,'RLerr':RLerr,'RT':RT,'RTerr':RTerr,'Chi2':Chi2,
                    'num_points':len(y),'xerr':nu_mid})             
          
            fit = append_row(fit,new_row)
            #____________________________________ end Rosenbluth fit _________________________________________________________
            
            if Rosenbluth_plot:
                #____________________________________ start Rosenbluth plot __________________________________________________
                fig1 = plt.figure(figsize=(12,6))
                plt.plot(x,linear_model(x,a_opt,b_opt),color='gray',label = 'y='+str(round(a_opt,3))
                        +'*x+'+str(round(b_opt,3))+'\nRL,RT:'+str(round(RL,3))+','+str(round(RT,3)))
                datasets = picked['dataSet'].unique()
                            
                plt.title('$Q^{3}_{center}$:'+str(qvcenter)+'   Ex:'+str(round(Ex,3))+'   nu:'+str(round(nu,3))+'    RL,RT error: '+round_sig_3(RLerr)+','+round_sig_3(RTerr)
                        +'   $\chi^2$:'+str(round(Chi2,1))+'   DoF:'+str(len(y)-2) +'   $\\frac{\chi^2}{DoF}$:'+str(round(Chi2/(len(y)-2),1)))

                for dataset in datasets:
                    picked_dataset = picked.loc[picked['dataSet']==dataset]
                    plt.errorbar(picked_dataset['epsilon'],picked_dataset['Hbc_Sig(GeV)'],yerr=picked_dataset['Hbc_error(GeV)'],
                                fmt ='.',capsize=3,markersize='3')
                    plt.scatter(picked_dataset['epsilon'],picked_dataset['Hbc_Sig(GeV)'],s=10, label = str(dataset)+' '+dataSet_to_name[dataset])

                plt.xlabel('epsilon')
                plt.ylabel('Hbc')
                plt.legend()
                #____________________________________ end Rosenbluth plot ___________________________________________________

    fit['Chi2_DoF']=fit['Chi2']/(fit['num_points']-2)

    ChristyBodekFit = pd.read_csv('Qvedges/'+f'Qvedge_{qvcenter}.csv',index_col = False)
    ChristyBodekFit.columns = ['qv','q2','ex','nu','RT','RL','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
    

    # Scale the Christy-Bodek fit by 20/6
    scaling_factor = 20 / 6
    ChristyBodekFit["RL"] *= scaling_factor
    ChristyBodekFit["RT"] *= scaling_factor
    ChristyBodekFit["RLQE"] *= scaling_factor
    ChristyBodekFit["RTQE"] *= scaling_factor
    ChristyBodekFit["RTE"] *= scaling_factor
    
    # Christy-Bodek fit RLRT total
    axs[0].plot(ChristyBodekFit['nu'],ChristyBodekFit["RL"],color='black',label="RL(total), RT(total) Christy-Bodek Fit", linestyle='solid')
    axs[1].plot(ChristyBodekFit['nu'],ChristyBodekFit["RT"],color='black', linestyle='solid')
    # Christy-Bodek fit RLRT QE only
    axs[0].plot(ChristyBodekFit['nu'],ChristyBodekFit["RLQE"],color='black',label="RL(QE), RT(QE+TE) Christy-Bodek Fit", linestyle='dotted') 
    axs[1].plot(ChristyBodekFit['nu'],ChristyBodekFit["RTQE"]+ChristyBodekFit["RTE"],color='black', linestyle='dotted')
    

    # RL this analysis
    axs[0].scatter(fit['nu'], fit['RL'],color='red',label='RL, RT this analysis',**our_scatter_setting)
    axs[0].errorbar(fit['nu'], fit['RL'], yerr = fit['RLerr'], color='red', **errorbar_setting)
    axs[0].set_title("$\mathbf{q}=$"+str(qvcenter))
    axs[0].set_xlabel('$\\nu (GeV)$')
    axs[0].set_ylabel('$R_L$')

    # RT this analysis
    axs[1].scatter(fit['nu'], fit['RT'],color='red',**our_scatter_setting)
    axs[1].errorbar(fit['nu'], fit['RT'], yerr = fit['RTerr'], color='red', **errorbar_setting)
    axs[1].set_title("$\mathbf{q}=$"+str(qvcenter))
    axs[1].set_xlabel('$\\nu (GeV)$')
    axs[1].set_ylabel('$R_T$')

    # These W (final state invariant mass) are put on plot for kinematic reference
    W_peaks = np.array([0.93,1.07,1.23]) 
    W_colors = ['darkorange','turquoise','slateblue']
    W_locations = 0.025-mass_nucleon+np.sqrt(qvcenter**2+W_peaks**2)
    RL_plot_height = RL_plot_heights[bin_index]
    RT_plot_height = RT_plot_heights[bin_index]
    for k in range(len(W_peaks)):
        location = W_locations[k]
        if location < qvcenter:
            axs[0].axvline(x=location, color = W_colors[k], linestyle='dashed')
            axs[1].axvline(x=location, color = W_colors[k], linestyle='dashed')
            if k == 0:
                axs[0].text(location, RL_plot_height*0.98, '\n$W=$'+str(W_peaks[k]),**text_setting) 
                axs[1].text(location, RT_plot_height*0.98, '\n$W=$'+str(W_peaks[k]),**text_setting)
            elif k == 1:
                axs[0].text(location, RL_plot_height*0.98, '\n\n$W=$'+str(W_peaks[k]),**text_setting)
                axs[1].text(location, RT_plot_height*0.98, '\n\n$W=$'+str(W_peaks[k]),**text_setting)
            else:
                axs[0].text(location, RL_plot_height*0.98, '\n\n\n$W=$'+str(W_peaks[k]), **text_setting)
                axs[1].text(location, RT_plot_height*0.98, '\n\n\n$W=$'+str(W_peaks[k]), **text_setting)
        
    axs[0].axvline(x=qvcenter, color = 'brown', linestyle='dashdot')
    axs[1].axvline(x=qvcenter, color = 'brown', linestyle='dashdot')
    axs[0].text(qvcenter*0.95, RL_plot_height*0.98, '$Q^2=0$\n'+f'$(\\nu={qvcenter})$' ,**text_setting)
    axs[1].text(qvcenter*0.95, RT_plot_height*0.98, '$Q^2=0$\n'+f'$(\\nu={qvcenter})$' ,**text_setting)

    axs[0].set_xlim(0.0, qvcenter*1.05)
    axs[1].set_xlim(0.0, qvcenter*1.05)    

    axs[0].set_ylim(None,RL_plot_heights[bin_index])
    axs[1].set_ylim(None,RT_plot_heights[bin_index])
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    fig.legend(loc='lower center', ncol=4)
    plt.show()


In [18]:
# read the data frame: it should contain columns {'A', 'Z', 'nu', 'E0', 'ThetaDeg', 'cross', 'error', 'dataSet'}
df = prepare_df("40Ca_WilliamsonData_updated_RawData.csv")
# path to Prof. Christy's Fortran code
fortran_folder = 'response/'


In [19]:
'''
Run the following code chunks to generate bin-centering corrections factors for data line by line.

'''

'\nRun the following code chunks to generate bin-centering corrections factors for data line by line.\n\n'

In [20]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data) analysis in nu
df['RL_fortran_qvcenter_nu']=0
df['RT_fortran_qvcenter_nu']=0
df['RL_fortran_qvdata_nu']=0
df['RT_fortran_qvdata_nu']=0

center_errors = []
data_errors = []
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        nu = row['nu']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(nu)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, nu:',qvcenter,nu)
            center_errors.append(index)
            continue
        ex_center = center_output[2]
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_nu']=RL_center
        df.loc[index,'RT_fortran_qvcenter_nu']=RT_center



        # calculate RL RT of data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(nu)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, nu:',qv,nu)
            data_errors.append(index)
            continue
        ex_data = data_output[2]
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)
        df.loc[index,'RL_fortran_qvdata_nu']=RL_data
        df.loc[index,'RT_fortran_qvdata_nu']=RT_data




index: 9 RLRT_center: 0.0362 0.0162 RLRT_data: 0.0353 0.017
index: 10 RLRT_center: 0.0308 0.016 RLRT_data: 0.0312 0.0169
index: 11 RLRT_center: 0.0278 0.0147 RLRT_data: 0.028 0.015
index: 12 RLRT_center: 0.025 0.0131 RLRT_data: 0.0247 0.0127
index: 13 RLRT_center: 0.0225 0.0114 RLRT_data: 0.0214 0.0102
index: 14 RLRT_center: 0.0176 0.00805 RLRT_data: 0.0151 0.00563
index: 15 RLRT_center: 1.14e-07 5.15e-08 RLRT_data: 5.78e-08 6.1e-08
index: 16 RLRT_center: 0.00527 2.21e-07 RLRT_data: 0.00635 2.04e-07
index: 17 RLRT_center: 0.024 0.0108 RLRT_data: 0.0243 0.0114
index: 18 RLRT_center: 0.0432 0.014 RLRT_data: 0.0432 0.0139
index: 19 RLRT_center: 0.0368 0.0135 RLRT_data: 0.037 0.0135
index: 20 RLRT_center: 0.0368 0.0157 RLRT_data: 0.037 0.0154
index: 21 RLRT_center: 0.0338 0.0166 RLRT_data: 0.0333 0.0155
index: 22 RLRT_center: 0.0308 0.016 RLRT_data: 0.0293 0.0141
index: 0 RLRT_center: 9.78e-09 7.83e-08 RLRT_data: 6.94e-09 9.13e-08
index: 1 RLRT_center: 1.64e-08 7.84e-08 RLRT_data: 1.17e-08

index: 127 RLRT_center: 0.0166 0.0257 RLRT_data: 0.0173 0.0255
index: 128 RLRT_center: 0.0165 0.0245 RLRT_data: 0.0173 0.0233
index: 129 RLRT_center: 0.016 0.0231 RLRT_data: 0.0165 0.021
index: 130 RLRT_center: 0.0151 0.021 RLRT_data: 0.0151 0.0179
index: 131 RLRT_center: 0.0147 0.02 RLRT_data: 0.0145 0.0167
index: 132 RLRT_center: 0.0131 0.0169 RLRT_data: 0.0123 0.0128
index: 133 RLRT_center: 0.0117 0.0144 RLRT_data: 0.0106 0.0103
index: 159 RLRT_center: 0.00472 0.0101 RLRT_data: 0.0056 0.0102
index: 160 RLRT_center: 0.00427 0.0112 RLRT_data: 0.00506 0.0107
index: 161 RLRT_center: 0.00373 0.0131 RLRT_data: 0.00433 0.0122
index: 162 RLRT_center: 0.00337 0.0149 RLRT_data: 0.00387 0.014
index: 167 RLRT_center: 0.0145 0.0197 RLRT_data: 0.0134 0.0238
index: 168 RLRT_center: 0.0112 0.0137 RLRT_data: 0.0116 0.0174
index: 169 RLRT_center: 0.00878 0.0102 RLRT_data: 0.00929 0.0122
index: 170 RLRT_center: 0.00692 0.00877 RLRT_data: 0.00721 0.00938
index: 171 RLRT_center: 0.00572 0.00885 RLRT_dat

index: 269 RLRT_center: 0.00425 0.0117 RLRT_data: 0.00428 0.0122
index: 270 RLRT_center: 0.00363 0.0139 RLRT_data: 0.00363 0.0161
index: 271 RLRT_center: 0.00343 0.0192 RLRT_data: 0.00348 0.0217
index: 272 RLRT_center: 0.00344 0.0244 RLRT_data: 0.00345 0.0261
index: 273 RLRT_center: 0.00345 0.0283 RLRT_data: 0.00343 0.0287
index: 274 RLRT_center: 0.0034 0.0295 RLRT_data: 0.00344 0.0293
index: 275 RLRT_center: 0.00333 0.0283 RLRT_data: 0.00344 0.0283
index: 276 RLRT_center: 0.00343 0.0262 RLRT_data: 0.00349 0.0268
index: 277 RLRT_center: 0.00435 0.0245 RLRT_data: 0.00362 0.0254
index: 279 RLRT_center: 7.28e-06 1.51e-09 RLRT_data: 2.21e-07 2.19e-11
index: 280 RLRT_center: 0.00074 0.0031 RLRT_data: 0.00051 0.00236
index: 281 RLRT_center: 0.00271 0.00999 RLRT_data: 0.0024 0.00915
index: 282 RLRT_center: 0.00522 0.0173 RLRT_data: 0.00521 0.0173
index: 283 RLRT_center: 0.00656 0.0192 RLRT_data: 0.00672 0.0192
index: 284 RLRT_center: 0.00415 0.0118 RLRT_data: 0.00418 0.0118
index: 285 RLRT_ce

In [21]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data), analysis in Ex
df['RL_fortran_qvcenter_ex']=0
df['RT_fortran_qvcenter_ex']=0
df['RL_fortran_qvdata_ex']=0
df['RT_fortran_qvdata_ex']=0

center_errors = []
data_errors = []
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        Ex = row['Ex']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(Ex)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, Ex:',qvcenter,Ex)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_ex']=RL_center
        df.loc[index,'RT_fortran_qvcenter_ex']=RT_center


        # calculate RL RT of data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(Ex)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, Ex:',qv,Ex)
            data_errors.append(index)
            continue
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data,'Ex:',Ex)
        df.loc[index,'RL_fortran_qvdata_ex']=RL_data
        df.loc[index,'RT_fortran_qvdata_ex']=RT_data



index: 9 RLRT_center: 0.0357 0.0164 RLRT_data: 0.0349 0.0174 Ex: 0.03893473391742204
index: 10 RLRT_center: 0.0302 0.0158 RLRT_data: 0.0306 0.0167 Ex: 0.04921047391742204
index: 11 RLRT_center: 0.0272 0.0144 RLRT_data: 0.0274 0.0147 Ex: 0.05476094391742204
index: 12 RLRT_center: 0.0245 0.0128 RLRT_data: 0.0242 0.0124 Ex: 0.05976910391742204
index: 13 RLRT_center: 0.022 0.0111 RLRT_data: 0.021 0.00996 Ex: 0.06473772391742204
index: 14 RLRT_center: 0.0173 0.0078 RLRT_data: 0.0148 0.00549 Ex: 0.07525682391742204
index: 15 RLRT_center: 0.0293 5.24e-08 RLRT_data: 0.03 6.22e-08 Ex: 0.003924667866503072
index: 16 RLRT_center: 0.00469 4.24e-06 RLRT_data: 0.00463 3.97e-06 Ex: 0.012326993866503072
index: 17 RLRT_center: 0.0272 0.0319 RLRT_data: 0.0269 0.0325 Ex: 0.01973654786650307
index: 18 RLRT_center: 0.0438 0.0149 RLRT_data: 0.0438 0.0149 Ex: 0.025026498866503072
index: 19 RLRT_center: 0.0359 0.0136 RLRT_data: 0.0361 0.0136 Ex: 0.029932384866503074
index: 20 RLRT_center: 0.0365 0.0161 RLRT_d

index: 62 RLRT_center: 1.26e-08 5.31e-08 RLRT_data: 7.01e-09 2.2e-08 Ex: 0.0024985026735790568
index: 63 RLRT_center: 0.000976 1.36e-07 RLRT_data: 0.000338 6.8e-08 Ex: 0.010544482673579056
index: 64 RLRT_center: 0.00278 0.00733 RLRT_data: 0.00184 0.00594 Ex: 0.020889312673579057
index: 65 RLRT_center: 0.00735 0.00932 RLRT_data: 0.00607 0.00835 Ex: 0.03123413267357906
index: 66 RLRT_center: 0.0117 0.0141 RLRT_data: 0.0111 0.0137 Ex: 0.04157896267357906
index: 67 RLRT_center: 0.0128 0.0183 RLRT_data: 0.0133 0.0187 Ex: 0.05134908267357906
index: 68 RLRT_center: 0.0139 0.0225 RLRT_data: 0.0158 0.0235 Ex: 0.06341804267357906
index: 69 RLRT_center: 0.0153 0.0246 RLRT_data: 0.0181 0.0252 Ex: 0.07261344267357905
index: 70 RLRT_center: 0.0163 0.0256 RLRT_data: 0.0193 0.0249 Ex: 0.08180885267357905
index: 77 RLRT_center: 9.41e-09 5.29e-08 RLRT_data: 1.23e-08 7.3e-08 Ex: 0.0021643837215083615
index: 78 RLRT_center: 2.51e-06 5.81e-08 RLRT_data: 4.24e-06 8.27e-08 Ex: 0.005858315721508361
index: 79 

index: 156 RLRT_center: 0.00778 0.0144 RLRT_data: 0.0077 0.0128 Ex: 0.18636280091240723
index: 157 RLRT_center: 0.00709 0.0129 RLRT_data: 0.00687 0.0111 Ex: 0.19688854091240723
index: 158 RLRT_center: 0.00655 0.0119 RLRT_data: 0.0062 0.0103 Ex: 0.20636280091240722
index: 163 RLRT_center: 2.67e-08 4.13e-09 RLRT_data: 1e-08 1.65e-09 Ex: 0.0028694704985106198
index: 164 RLRT_center: 0.00257 0.00515 RLRT_data: 0.00258 0.00515 Ex: 0.03746411949851062
index: 165 RLRT_center: 0.00542 0.0142 RLRT_data: 0.00694 0.0165 Ex: 0.06821974949851062
index: 166 RLRT_center: 0.00869 0.0215 RLRT_data: 0.0119 0.0246 Ex: 0.09630444349851063
index: 193 RLRT_center: 0.00721 0.0131 RLRT_data: 0.00715 0.0162 Ex: 0.19468330169023543
index: 194 RLRT_center: 0.00554 0.011 RLRT_data: 0.00563 0.0114 Ex: 0.22549411169023545
index: 195 RLRT_center: 0.00478 0.0117 RLRT_data: 0.00477 0.0117 Ex: 0.24603465269023544
index: 196 RLRT_center: 0.00414 0.0139 RLRT_data: 0.00406 0.0147 Ex: 0.26603465269023546
index: 197 RLRT_ce

index: 278 RLRT_center: 0.00389 0.0243 RLRT_data: 0.00389 0.0242 Ex: 0.5289543002294432
index: 290 RLRT_center: 0.00371 0.026 RLRT_data: 0.00361 0.027 Ex: 0.4808208993934997
index: 291 RLRT_center: 0.00379 0.0254 RLRT_data: 0.0037 0.0256 Ex: 0.5065375303934997
index: 292 RLRT_center: 0.00391 0.0242 RLRT_data: 0.00389 0.0241 Ex: 0.5322505923934997
index: 293 RLRT_center: 0.00422 0.0232 RLRT_data: 0.00422 0.0232 Ex: 0.5546116393934997
index: 294 RLRT_center: 0.00503 0.0229 RLRT_data: 0.00472 0.0227 Ex: 0.5747308693934997
index: 295 RLRT_center: 5.56e-05 0.000355 RLRT_data: 6.18e-05 0.000383 Ex: 0.029582105723198734
index: 296 RLRT_center: 0.000343 0.00205 RLRT_data: 0.000497 0.00269 Ex: 0.07345006472319873
index: 297 RLRT_center: 0.00154 0.00702 RLRT_data: 0.00241 0.00981 Ex: 0.11978185672319874
index: 298 RLRT_center: 0.00317 0.0127 RLRT_data: 0.00457 0.0164 Ex: 0.16052504472319873
index: 302 RLRT_center: 0.00309 0.0131 RLRT_data: 0.00332 0.0156 Ex: 0.34471759872319874
index: 303 RLRT_c

In [22]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in nu
df['RL_fortran_q2center_nu']=0
df['RT_fortran_q2center_nu']=0
df['RL_fortran_q2data_nu']=0
df['RT_fortran_q2data_nu']=0

center_errors = []
data_errors = []
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        nu = row['nu']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(nu)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, nu:',Q2center,nu)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_nu']=RL_center
        df.loc[index,'RT_fortran_q2center_nu']=RT_center

        # calculate RL RT of data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(nu)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, nu:',Q2eff,nu)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_nu']=RL_data
        df.loc[index,'RT_fortran_q2data_nu']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 14 RLRT_center: 0.0103 0.00213 RLRT_data: 0.0151 0.00563
index done: 11 RLRT_center: 0.0221 0.00901 RLRT_data: 0.028 0.015
index done: 12 RLRT_center: 0.0194 0.0076 RLRT_data: 0.0247 0.0127
index done: 13 RLRT_center: 0.0171 0.00634 RLRT_data: 0.0214 0.0102
index done: 20 RLRT_center: 0.0344 0.0129 RLRT_data: 0.037 0.0154
index done: 21 RLRT_center: 0.0297 0.0122 RLRT_data: 0.0333 0.0155
index done: 22 RLRT_center: 0.0255 0.0106 RLRT_data: 0.0293 0.0141
index done: 9 RLRT_center: 0.0362 0.0162 RLRT_data: 0.0353 0.017
index done: 10 RLRT_center: 0.0309 0.0162 RLRT_data: 0.0312 0.0169
index done: 15 RLRT_center: 2.48e-07 4.97e-08 RLRT_data: 5.78e-08 6.1e-08
index done: 16 RLRT_center: 0.00437 2.35e-07 RLRT_data: 0.00635 2.04e-07
index done: 17 RLRT_center: 0.0233 0.0103 RLRT_data: 0.0243 0.0114
index done: 18 RLRT_center: 0.044 0.0143 RLRT_data: 0.0432 0.0139
index done: 19 RLRT_center: 0.037 0.0135 RLRT_data: 0.037 0.0135
index done: 35 RLRT_center: 0.0162 0.00763 RLRT_data:

index done: 42 RLRT_center: 0.00715 0.00659 RLRT_data: 0.00797 0.00696
index done: 43 RLRT_center: 0.0123 0.0126 RLRT_data: 0.0149 0.0138
index done: 65 RLRT_center: 0.00936 0.00968 RLRT_data: 0.00441 0.00639
index done: 66 RLRT_center: 0.0148 0.015 RLRT_data: 0.00955 0.0117
index done: 67 RLRT_center: 0.0168 0.0193 RLRT_data: 0.0132 0.017
index done: 68 RLRT_center: 0.017 0.0232 RLRT_data: 0.0153 0.0225
index done: 69 RLRT_center: 0.0179 0.0249 RLRT_data: 0.0177 0.0249
index done: 70 RLRT_center: 0.0185 0.0254 RLRT_data: 0.0193 0.0252
index done: 77 RLRT_center: 1.34e-09 8.33e-08 RLRT_data: 9.38e-10 7.24e-08
index done: 78 RLRT_center: 1.31e-08 8.38e-08 RLRT_data: 9.78e-09 7.52e-08
index done: 79 RLRT_center: 0.000814 1.12e-07 RLRT_data: 0.000704 1.03e-07
index done: 80 RLRT_center: 0.0118 6.59e-06 RLRT_data: 0.0119 6.46e-06
index done: 81 RLRT_center: 0.00739 0.00689 RLRT_data: 0.00838 0.00734
index done: 82 RLRT_center: 0.0136 0.0138 RLRT_data: 0.0162 0.0151
index done: 98 RLRT_cent

index done: 107 RLRT_center: 0.000677 0.00197 RLRT_data: 0.000692 0.002
index done: 108 RLRT_center: 0.0017 0.00497 RLRT_data: 0.00254 0.00665
index done: 109 RLRT_center: 0.00274 0.00887 RLRT_data: 0.00517 0.0138
index done: 142 RLRT_center: 0.00115 0.00301 RLRT_data: 0.000256 0.00108
index done: 143 RLRT_center: 0.00165 0.0047 RLRT_data: 0.000518 0.00205
index done: 144 RLRT_center: 0.00209 0.00676 RLRT_data: 0.000884 0.0036
index done: 145 RLRT_center: 0.00273 0.00885 RLRT_data: 0.00146 0.00564
index done: 146 RLRT_center: 0.0036 0.0114 RLRT_data: 0.00251 0.00881
index done: 147 RLRT_center: 0.00437 0.0136 RLRT_data: 0.00366 0.012
index done: 148 RLRT_center: 0.00513 0.0157 RLRT_data: 0.00505 0.0155
index done: 149 RLRT_center: 0.00586 0.0177 RLRT_data: 0.00656 0.019
index done: 150 RLRT_center: 0.00647 0.0192 RLRT_data: 0.00798 0.0216
index done: 151 RLRT_center: 0.00696 0.0204 RLRT_data: 0.00913 0.023
index done: 163 RLRT_center: 6.73e-12 8.03e-10 RLRT_data: 1.58e-11 1.63e-09
inde

In [23]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in Ex
df['RL_fortran_q2center_ex']=0
df['RT_fortran_q2center_ex']=0
df['RL_fortran_q2data_ex']=0
df['RT_fortran_q2data_ex']=0

center_errors = []
data_errors = []
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        Ex = row['Ex']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(Ex)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, Ex:',Q2center,Ex)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_ex']=RL_center
        df.loc[index,'RT_fortran_q2center_ex']=RT_center


        # calculate RL RT of data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(Ex)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, Ex:',Q2eff,Ex)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_ex']=RL_data
        df.loc[index,'RT_fortran_q2data_ex']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 14 RLRT_center: 0.0102 0.00211 RLRT_data: 0.0149 0.00552
index done: 11 RLRT_center: 0.0219 0.00889 RLRT_data: 0.0275 0.0147
index done: 12 RLRT_center: 0.0192 0.00749 RLRT_data: 0.0242 0.0124
index done: 13 RLRT_center: 0.0169 0.00624 RLRT_data: 0.0211 0.01
index done: 20 RLRT_center: 0.0341 0.0129 RLRT_data: 0.0367 0.0156
index done: 21 RLRT_center: 0.0292 0.012 RLRT_data: 0.0327 0.0155
index done: 22 RLRT_center: 0.0251 0.0105 RLRT_data: 0.0288 0.0138
index done: 9 RLRT_center: 0.0357 0.0163 RLRT_data: 0.0349 0.0174
index done: 10 RLRT_center: 0.0303 0.016 RLRT_data: 0.0307 0.0167
index done: 15 RLRT_center: 0.0288 5.05e-08 RLRT_data: 0.0299 6.22e-08
index done: 16 RLRT_center: 0.0047 4.4e-06 RLRT_data: 0.00463 3.96e-06
index done: 17 RLRT_center: 0.0275 0.0313 RLRT_data: 0.0269 0.0326
index done: 18 RLRT_center: 0.0444 0.0151 RLRT_data: 0.0437 0.0149
index done: 19 RLRT_center: 0.0361 0.0136 RLRT_data: 0.0361 0.0136
index done: 35 RLRT_center: 0.016 0.00749 RLRT_data: 0

index done: 42 RLRT_center: 0.00907 0.00927 RLRT_data: 0.00978 0.00956
index done: 43 RLRT_center: 0.0143 0.0145 RLRT_data: 0.0167 0.0156
index done: 65 RLRT_center: 0.011 0.0114 RLRT_data: 0.00603 0.00832
index done: 66 RLRT_center: 0.0159 0.0166 RLRT_data: 0.011 0.0137
index done: 67 RLRT_center: 0.0168 0.0206 RLRT_data: 0.0133 0.0186
index done: 68 RLRT_center: 0.0173 0.024 RLRT_data: 0.0157 0.0234
index done: 69 RLRT_center: 0.0182 0.0252 RLRT_data: 0.018 0.0252
index done: 70 RLRT_center: 0.0185 0.0253 RLRT_data: 0.0192 0.025
index done: 77 RLRT_center: 1.43e-08 8.39e-08 RLRT_data: 1.23e-08 7.29e-08
index done: 78 RLRT_center: 5.21e-06 9.19e-08 RLRT_data: 4.31e-06 8.26e-08
index done: 79 RLRT_center: 0.000354 3.22e-07 RLRT_data: 0.000335 3.04e-07
index done: 80 RLRT_center: 0.0059 0.00279 RLRT_data: 0.00582 0.00277
index done: 81 RLRT_center: 0.00918 0.00944 RLRT_data: 0.01 0.00978
index done: 82 RLRT_center: 0.0153 0.0157 RLRT_data: 0.0177 0.0167
index done: 98 RLRT_center: 0.018

index done: 109 RLRT_center: 0.0034 0.0108 RLRT_data: 0.00588 0.0156
index done: 142 RLRT_center: 0.00152 0.00414 RLRT_data: 0.000409 0.00173
index done: 143 RLRT_center: 0.00192 0.00609 RLRT_data: 0.000696 0.003
index done: 144 RLRT_center: 0.00257 0.00835 RLRT_data: 0.00121 0.0049
index done: 145 RLRT_center: 0.0033 0.0105 RLRT_data: 0.00193 0.00722
index done: 146 RLRT_center: 0.00419 0.0131 RLRT_data: 0.00307 0.0105
index done: 147 RLRT_center: 0.00493 0.0152 RLRT_data: 0.00423 0.0137
index done: 148 RLRT_center: 0.00564 0.0171 RLRT_data: 0.00557 0.017
index done: 149 RLRT_center: 0.00629 0.0188 RLRT_data: 0.00697 0.02
index done: 150 RLRT_center: 0.00682 0.0201 RLRT_data: 0.00823 0.0221
index done: 151 RLRT_center: 0.00721 0.0209 RLRT_data: 0.00919 0.023
index done: 163 RLRT_center: 4.62e-09 8.09e-10 RLRT_data: 9.63e-09 1.64e-09
index done: 164 RLRT_center: 0.00119 0.00311 RLRT_data: 0.00255 0.00512
index done: 187 RLRT_center: 0.00672 0.0198 RLRT_data: 0.00383 0.0138
index done: 

In [24]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data) analysis in W2
df['RL_fortran_qvcenter_w2']=0
df['RT_fortran_qvcenter_w2']=0
df['RL_fortran_qvdata_w2']=0
df['RT_fortran_qvdata_w2']=0

center_errors = []
data_errors = []
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        W2 = row['W2']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(W2)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, W2:',qvcenter,W2)
            center_errors.append(index)
            continue
        ex_center = center_output[2]
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_w2']=RL_center
        df.loc[index,'RT_fortran_qvcenter_w2']=RT_center



        # calculate RL RT of data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(W2)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, W2:',qv,W2)
            data_errors.append(index)
            continue
        ex_data = data_output[2]
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)
        df.loc[index,'RL_fortran_qvdata_w2']=RL_data
        df.loc[index,'RT_fortran_qvdata_w2']=RT_data




index: 9 RLRT_center: 0.0368 0.0153 RLRT_data: 0.0353 0.017
index: 10 RLRT_center: 0.0315 0.0163 RLRT_data: 0.0312 0.0169
index: 11 RLRT_center: 0.028 0.0148 RLRT_data: 0.028 0.015
index: 12 RLRT_center: 0.0248 0.0129 RLRT_data: 0.0247 0.0127
index: 13 RLRT_center: 0.0218 0.0109 RLRT_data: 0.0214 0.0102
index: 14 RLRT_center: 0.0163 0.00716 RLRT_data: 0.0151 0.00563
index: 15 RLRT_center: 6.82e-09 5.1e-08 RLRT_data: 5.78e-08 6.1e-08
index: 16 RLRT_center: 0.00845 9.07e-08 RLRT_data: 0.00635 2.04e-07
index: 17 RLRT_center: 0.0114 0.00648 RLRT_data: 0.0243 0.0114
index: 18 RLRT_center: 0.0431 0.0139 RLRT_data: 0.0432 0.0139
index: 19 RLRT_center: 0.0362 0.0135 RLRT_data: 0.037 0.0135
index: 20 RLRT_center: 0.0364 0.0161 RLRT_data: 0.037 0.0154
index: 21 RLRT_center: 0.0326 0.0165 RLRT_data: 0.0333 0.0155
index: 22 RLRT_center: 0.0293 0.0154 RLRT_data: 0.0293 0.0141
index: 0 RLRT_center: 4.39e-10 7.81e-08 RLRT_data: 6.94e-09 9.13e-08
index: 1 RLRT_center: 8.72e-10 7.81e-08 RLRT_data: 1.17

index: 125 RLRT_center: 0.0138 0.0222 RLRT_data: 0.0134 0.0224
index: 126 RLRT_center: 0.0158 0.0252 RLRT_data: 0.0159 0.0252
index: 127 RLRT_center: 0.0167 0.0256 RLRT_data: 0.0173 0.0255
index: 128 RLRT_center: 0.0162 0.0236 RLRT_data: 0.0173 0.0233
index: 129 RLRT_center: 0.0153 0.0215 RLRT_data: 0.0165 0.021
index: 130 RLRT_center: 0.014 0.0187 RLRT_data: 0.0151 0.0179
index: 131 RLRT_center: 0.0135 0.0177 RLRT_data: 0.0145 0.0167
index: 132 RLRT_center: 0.0116 0.0142 RLRT_data: 0.0123 0.0128
index: 133 RLRT_center: 0.0101 0.0119 RLRT_data: 0.0106 0.0103
index: 159 RLRT_center: 0.00566 0.00887 RLRT_data: 0.0056 0.0102
index: 160 RLRT_center: 0.00494 0.00962 RLRT_data: 0.00506 0.0107
index: 161 RLRT_center: 0.00419 0.0115 RLRT_data: 0.00433 0.0122
index: 162 RLRT_center: 0.00365 0.0135 RLRT_data: 0.00387 0.014
index: 167 RLRT_center: 0.0163 0.0237 RLRT_data: 0.0134 0.0238
index: 168 RLRT_center: 0.0128 0.0165 RLRT_data: 0.0116 0.0174
index: 169 RLRT_center: 0.0097 0.0114 RLRT_data: 

index: 266 RLRT_center: 0.00622 0.0195 RLRT_data: 0.00734 0.0211
index: 267 RLRT_center: 0.00622 0.0172 RLRT_data: 0.00729 0.0179
index: 268 RLRT_center: 0.00486 0.0126 RLRT_data: 0.00546 0.0124
index: 269 RLRT_center: 0.00394 0.0121 RLRT_data: 0.00428 0.0122
index: 270 RLRT_center: 0.0035 0.0156 RLRT_data: 0.00363 0.0161
index: 271 RLRT_center: 0.00346 0.0209 RLRT_data: 0.00348 0.0217
index: 272 RLRT_center: 0.00346 0.0254 RLRT_data: 0.00345 0.0261
index: 273 RLRT_center: 0.00346 0.0285 RLRT_data: 0.00343 0.0287
index: 274 RLRT_center: 0.00341 0.0295 RLRT_data: 0.00344 0.0293
index: 275 RLRT_center: 0.00334 0.0287 RLRT_data: 0.00344 0.0283
index: 276 RLRT_center: 0.00333 0.0272 RLRT_data: 0.00349 0.0268
index: 277 RLRT_center: 0.00357 0.0256 RLRT_data: 0.00362 0.0254
index: 279 RLRT_center: 1.12e-12 4.95e-11 RLRT_data: 2.21e-07 2.19e-11
index: 280 RLRT_center: 0.000535 0.00205 RLRT_data: 0.00051 0.00236
index: 281 RLRT_center: 0.00244 0.00914 RLRT_data: 0.0024 0.00915
index: 282 RLRT_

In [25]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in W2
df['RL_fortran_q2center_w2']=0
df['RT_fortran_q2center_w2']=0
df['RL_fortran_q2data_w2']=0
df['RT_fortran_q2data_w2']=0

center_errors = []
data_errors = []
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate RL RT at center
        W2 = row['W2']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(W2)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, W2:',Q2center,W2)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_w2']=RL_center
        df.loc[index,'RT_fortran_q2center_w2']=RT_center


        # calculate RL RT of data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(W2)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, W2:',Q2eff,W2)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_w2']=RL_data
        df.loc[index,'RT_fortran_q2data_w2']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 14 RLRT_center: 0.0116 0.0028 RLRT_data: 0.0151 0.00563
index done: 11 RLRT_center: 0.0267 0.0111 RLRT_data: 0.028 0.015
index done: 12 RLRT_center: 0.0227 0.00931 RLRT_data: 0.0247 0.0127
index done: 13 RLRT_center: 0.0193 0.00754 RLRT_data: 0.0214 0.0102
index done: 20 RLRT_center: 0.0352 0.0122 RLRT_data: 0.037 0.0154
index done: 21 RLRT_center: 0.0337 0.0129 RLRT_data: 0.0333 0.0155
index done: 22 RLRT_center: 0.0284 0.0118 RLRT_data: 0.0293 0.0141
index done: 9 RLRT_center: 0.0368 0.015 RLRT_data: 0.0353 0.017
index done: 10 RLRT_center: 0.0315 0.0164 RLRT_data: 0.0312 0.0169
index done: 15 RLRT_center: 3.22e-09 4.91e-08 RLRT_data: 5.78e-08 6.1e-08
index done: 16 RLRT_center: 0.00662 7.08e-08 RLRT_data: 0.00635 2.04e-07
index done: 17 RLRT_center: 0.0116 0.00662 RLRT_data: 0.0243 0.0114
index done: 18 RLRT_center: 0.0422 0.0132 RLRT_data: 0.0432 0.0139
index done: 19 RLRT_center: 0.037 0.0135 RLRT_data: 0.037 0.0135
index done: 35 RLRT_center: 0.0179 0.00876 RLRT_data:

index done: 40 RLRT_center: 0.0237 8.61e-08 RLRT_data: 0.000538 9.72e-08
index done: 41 RLRT_center: 0.00257 2.3e-07 RLRT_data: 0.00621 2.67e-06
index done: 42 RLRT_center: 0.00848 0.00836 RLRT_data: 0.00797 0.00696
index done: 43 RLRT_center: 0.0151 0.0154 RLRT_data: 0.0149 0.0138
index done: 65 RLRT_center: 0.017 1.64e-07 RLRT_data: 0.00441 0.00639
index done: 66 RLRT_center: 0.00795 0.0076 RLRT_data: 0.00955 0.0117
index done: 67 RLRT_center: 0.0148 0.015 RLRT_data: 0.0132 0.017
index done: 68 RLRT_center: 0.0167 0.022 RLRT_data: 0.0153 0.0225
index done: 69 RLRT_center: 0.0178 0.0248 RLRT_data: 0.0177 0.0249
index done: 70 RLRT_center: 0.0185 0.0253 RLRT_data: 0.0193 0.0252
index done: 77 RLRT_center: 6.61e-11 8.32e-08 RLRT_data: 9.38e-10 7.24e-08
index done: 78 RLRT_center: 1.46e-09 8.33e-08 RLRT_data: 9.78e-09 7.52e-08
index done: 79 RLRT_center: 6.28e-06 9.18e-08 RLRT_data: 0.000704 1.03e-07
index done: 80 RLRT_center: 0.0129 5.67e-06 RLRT_data: 0.0119 6.46e-06
index done: 81 RL

index done: 93 RLRT_center: 0.000273 0.00116 RLRT_data: 8.48e-10 4.52e-09
index done: 106 RLRT_center: 2.69e-18 7.99e-10 RLRT_data: 1.09e-12 1.65e-10
index done: 107 RLRT_center: 0.000704 0.00204 RLRT_data: 0.000692 0.002
index done: 108 RLRT_center: 0.00225 0.00734 RLRT_data: 0.00254 0.00665
index done: 109 RLRT_center: 0.0045 0.014 RLRT_data: 0.00517 0.0138
index done: 142 RLRT_center: 9.26e-11 8.01e-10 RLRT_data: 0.000256 0.00108
index done: 143 RLRT_center: 0.00015 0.000112 RLRT_data: 0.000518 0.00205
index done: 144 RLRT_center: 0.000861 0.00238 RLRT_data: 0.000884 0.0036
index done: 145 RLRT_center: 0.00165 0.00472 RLRT_data: 0.00146 0.00564
index done: 146 RLRT_center: 0.00259 0.00841 RLRT_data: 0.00251 0.00881
index done: 147 RLRT_center: 0.0038 0.012 RLRT_data: 0.00366 0.012
index done: 148 RLRT_center: 0.00508 0.0156 RLRT_data: 0.00505 0.0155
index done: 149 RLRT_center: 0.00621 0.0186 RLRT_data: 0.00656 0.019
index done: 150 RLRT_center: 0.00704 0.0206 RLRT_data: 0.00798 0.0

In [26]:
# calculate all bin centering correction factors Ex, nu, W2 in qv or q2 bins
# important formulas:
# qv*qv = Q2 + nu*nu
# Q2 = qv*qv - nu*nu

# bin centering correction for qv bin in Ex:
df["bc_qv_ex"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
                df.loc[index, 'bc_qv_ex']= (row['epsilon']*row['RL_fortran_qvcenter_ex']+0.5*((qvcenter**2
                                                        )/(qvcenter**2-row['nu']**2)
                                                    )*row['RT_fortran_qvcenter_ex']
                    )/(row['epsilon']*row['RL_fortran_qvdata_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_ex'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
                      'Ex:',row['Ex'],'RT_fortran_qvdata_ex:',row['RT_fortran_qvdata_ex'])
            

# bin centering correction for qv bin in nu:
df["bc_qv_nu"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
               df.loc[index, 'bc_qv_nu']= (row['epsilon']*row['RL_fortran_qvcenter_nu']+0.5*((qvcenter**2
                                                        )/(qvcenter**2-row['nu']**2)
                                                    )*row['RT_fortran_qvcenter_nu']
                    )/(row['epsilon']*row['RL_fortran_qvdata_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_nu'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
                      'Ex:',row['Ex'],'RT_fortran_qvdata_nu:',row['RT_fortran_qvdata_nu'])
                

# bin centering correction for qv bin in W2:
df["bc_qv_w2"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
               df.loc[index, 'bc_qv_w2']= (row['epsilon']*row['RL_fortran_qvcenter_w2']+0.5*((qvcenter**2
                                                        )/(
                                                            2*mass_nucleon*np.sqrt(qvcenter**2+row['W2'])-row['W2']-mass_nucleon**2
                                                        )
                                                    )*row['RT_fortran_qvcenter_w2']
                    )/(row['epsilon']*row['RL_fortran_qvdata_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_w2'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)


# bin centering correction for q2 bin in Ex:
df["bc_q2_ex"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        df.loc[index, 'bc_q2_ex']= (row['epsilon']*row['RL_fortran_q2center_ex']+0.5*((Q2center+row["nu"]**2
                                                    )/(Q2center)
                                                )*row['RT_fortran_q2center_ex']
                )/(row['epsilon']*row['RL_fortran_q2data_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_ex'])
        
# bin centering correction for q2 bin in nu:
df["bc_q2_nu"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        df.loc[index, 'bc_q2_nu']= (row['epsilon']*row['RL_fortran_q2center_nu']+0.5*((Q2center+row["nu"]**2
                                                    )/(Q2center)
                                                )*row['RT_fortran_q2center_nu']
                )/(row['epsilon']*row['RL_fortran_q2data_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_nu'])

# bin centering correction for q2 bin in W2:
df["bc_q2_w2"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        nu = (row['W2']+Q2center-mass_nucleon**2)/(2*mass_nucleon)
        qv2center = Q2center+nu**2
        df.loc[index, 'bc_q2_w2']= (row['epsilon']*row['RL_fortran_q2center_w2']+0.5*(
                                                        qv2center/(Q2center)
                                                )*row['RT_fortran_q2center_w2']
                )/(row['epsilon']*row['RL_fortran_q2data_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_w2'])


In [27]:
# calculate all bin centering correction factors Ex, nu, W2 in qv or q2 bins
# important formulas:
# qv*qv = Q2 + nu*nu
# Q2 = qv*qv - nu*nu

# bin centering correction for qv bin in Ex:
df["bc_qv_ex"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]

    #update 'bc_qv_ex', epsilonA, etc 
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN'):
            if row['Ex'] <= 0.025:
                continue
            try:
                denominator = row['epsilon'] * row['RL_fortran_qvdata_ex'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_qvdata_ex']
                if denominator == 0:
                    print(f"ZeroDivisionError, index: {index}, qvcenter: {qvcenter}, Q2eff: {row['Q2eff']}")
                    continue
                df.loc[index, 'bc_qv_ex'] = (row['epsilon'] * row['RL_fortran_qvcenter_ex'] + 0.5 * ((qvcenter ** 2) / (qvcenter ** 2 - row['nu'] ** 2)) * row['RT_fortran_qvcenter_ex']) / denominator
            except ZeroDivisionError:
                print('ZeroDivisionError, index:', index, 'qvcenter:', qvcenter, 'nu:', row['nu'], 'Ex:', row['Ex'], 'RT_fortran_qvdata_ex:', row['RT_fortran_qvdata_ex'])

# bin centering correction for qv bin in nu:
df["bc_qv_nu"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN'):
            if row['Ex'] <= 0.025:
                continue
            try:
                denominator = row['epsilon'] * row['RL_fortran_qvdata_nu'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_qvdata_nu']
                if denominator == 0:
                    print(f"ZeroDivisionError, index: {index}, qvcenter: {qvcenter}, Q2eff: {row['Q2eff']}")
                    continue
                df.loc[index, 'bc_qv_nu'] = (row['epsilon'] * row['RL_fortran_qvcenter_nu'] + 0.5 * ((qvcenter ** 2) / (qvcenter ** 2 - row['nu'] ** 2)) * row['RT_fortran_qvcenter_nu']) / denominator
            except ZeroDivisionError:
                print('ZeroDivisionError, index:', index, 'qvcenter:', qvcenter, 'nu:', row['nu'], 'Ex:', row['Ex'], 'RT_fortran_qvdata_nu:', row['RT_fortran_qvdata_nu'])

# bin centering correction for qv bin in W2:
df["bc_qv_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN'):
            if row['Ex'] <= 0.025:
                continue
            try:
                denominator = row['epsilon'] * row['RL_fortran_qvdata_w2'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_qvdata_w2']
                if denominator == 0:
                    print(f"ZeroDivisionError, index: {index}, qvcenter: {qvcenter}, Q2eff: {row['Q2eff']}")
                    continue
                df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_fortran_qvcenter_w2'] + 0.5 * ((qvcenter ** 2) / (2 * mass_nucleon * np.sqrt(qvcenter ** 2 + row['W2']) - row['W2'] - mass_nucleon ** 2)) * row['RT_fortran_qvcenter_w2']) / denominator
            except ZeroDivisionError:
                print('ZeroDivisionError, index:', index)

# bin centering correction for q2 bin in Ex:
df["bc_q2_ex"] = 1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]

    for index, row in picked.iterrows():
        if row['Ex'] <= 0.025:
            continue
        try:
            denominator = row['epsilon'] * row['RL_fortran_q2data_ex'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_q2data_ex']
            if denominator == 0:
                print(f"ZeroDivisionError, index: {index}, Q2center: {Q2center}, Q2eff: {row['Q2eff']}")
                continue
            df.loc[index, 'bc_q2_ex'] = (row['epsilon'] * row['RL_fortran_q2center_ex'] + 0.5 * ((Q2center + row["nu"] ** 2) / Q2center) * row['RT_fortran_q2center_ex']) / denominator
        except ZeroDivisionError:
            print('ZeroDivisionError, index:', index, 'Q2center:', Q2center, 'nu:', row['nu'], 'Ex:', row['Ex'], 'RT_fortran_q2data_ex:', row['RT_fortran_q2data_ex'])

# bin centering correction for q2 bin in nu:
df["bc_q2_nu"] = 1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]

    for index, row in picked.iterrows():
        if row['Ex'] <= 0.025:
            continue
        try:
            denominator = row['epsilon'] * row['RL_fortran_q2data_nu'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_q2data_nu']
            if denominator == 0:
                print(f"ZeroDivisionError, index: {index}, Q2center: {Q2center}, Q2eff: {row['Q2eff']}")
                continue
            df.loc[index, 'bc_q2_nu'] = (row['epsilon'] * row['RL_fortran_q2center_nu'] + 0.5 * ((Q2center + row["nu"] ** 2) / Q2center) * row['RT_fortran_q2center_nu']) / denominator
        except ZeroDivisionError:
            print('ZeroDivisionError, index:', index, 'Q2center:', Q2center, 'nu:', row['nu'], 'Ex:', row['Ex'], 'RT_fortran_q2data_nu:', row['RT_fortran_q2data_nu'])

# bin centering correction for q2 bin in W2:
df["bc_q2_w2"] = 1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]

    for index, row in picked.iterrows():
        if row['Ex'] <= 0.025:
            continue
        nu = (row['W2'] + Q2center - mass_nucleon ** 2) / (2 * mass_nucleon)
        qv2center = Q2center + nu ** 2
        try:
            denominator = row['epsilon'] * row['RL_fortran_q2data_w2'] + 0.5 * (row['q3momt_squared'] / row['Q2eff']) * row['RT_fortran_q2data_w2']
            if denominator == 0:
                print(f"ZeroDivisionError, index: {index}, Q2center: {Q2center}, Q2eff: {row['Q2eff']}")
                continue
            df.loc[index, 'bc_q2_w2'] = (row['epsilon'] * row['RL_fortran_q2center_w2'] + 0.5 * (qv2center / Q2center) * row['RT_fortran_q2center_w2']) / denominator
        except ZeroDivisionError:
            print('ZeroDivisionError, index:', index)


In [28]:
# save the dataframe
df.to_csv('40Ca_WilliamsonData_updated_BinCentering.csv',index=False)
